In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

from tqdm import tqdm

import get_training_data

In [ ]:
data_dir = '../data/sampling_locations/'

collection = 'S2L1C'
clear_threshold = 0.75  # minimum clear pixels for a tile to be included in the dataset
patch_size = 32

#start_dates = ['2020-01-01', '2021-01-01', '2022-01-01']
#end_dates = ['2021-01-01', '2022-01-01', '2023-01-01']
start_dates = ['2019-01-01']
end_dates = ['2020-01-01']


data_files = ['bolivar_2020_thresh_0.8_1_negatives.geojson',
              'amazonas_2020_thresh_0.5_2_negatives.geojson',
              'riverbank_negatives.geojson',
              'amazonas_2020_thresh_0.8_sumbsample_3_positives.geojson',
              'MinesPos2018-2020Sentinel_points.geojson',
              'bolivar_2020_thresh_0.8_sumbsample_5_positives.geojson',
              'full_amazon_v9_negatives.geojson',
              'v2.0_bolivar_negatives.geojson',
              'v2.1.1_bolivar_negatives.geojson',
              'v2.4_amazonas_negatives.geojson',
              'v2.4_amazon_negatives.geojson',
              'v2.4_amazon_positives.geojson',
              'v2.6_amazon_thresh_0.8_negatives.geojson',
              'v2.6_amazon_negatives.geojson',
              'v2.6_amazon_negatives_v2.geojson'
              ]

data_file_labels = [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]

data_files = [os.path.join(data_dir, f) for f in data_files]

# create the training data
for sampling_file, label_class in tqdm(zip(data_files, data_file_labels)):
    print(f'Creating training data for {sampling_file} with class {label_class}')
    for start_date, end_date in zip(start_dates, end_dates):
        extractor = get_training_data.TrainingData(patch_size, collection, clear_threshold)
        extractor.get_patches(sampling_file, label_class, start_date, end_date)
